In [6]:
from __future__ import division, print_function

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D, UpSampling2D, Conv2D, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1   #imagens passadas em grayscale com apenas um canal
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100   #dimensao do espaco latente (ruido)
        
        optimizer = Adam(0.0002, 0.5)   # taxa de aprendizado e beta1 do otimizador Adam
        
        # Criando e compilando o discriminador
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss = 'binary_crossentropy',
                                  optimizer = optimizer,
                                  metrics = ['accuracy'])
        
        # Construindo e compilando o gerador
        self.generator = self.build_generator()
        
        # Gerador recebe ruido na entrada e gera imagens
        z = Input(shape=(self.latent_dim))
        img = self.generator(z)
        
        # Somente o gerador e treinado no modelo da GAN
        self.discriminator.trainable = False
        
        # O discriminador recebe imagens geradas e determina sua validade
        validity = self.discriminator(img)
        
        # O modelo combinado treina o gerador com base na resposta do discriminador
        self.combined = Model(z, validity)
        self.combined.compile(loss = 'binary_crossentropy',
                             optimizer = optimizer)
        
        
    def build_generator(self):
        
        model = Sequential()
        
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))
        
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)
        
        return Model(noise, img)
    
    def build_discriminator(self):
        
        model = Sequential()
        
        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        
        model.summary()
        
        img = Input(shape=self.img_shape)
        validity = model(img)
        
        return Model(img, validity)
    
    
    def train(self, epochs, batch_size=128, sample_interval=50):
        
        (X_train, _),(_, _) = mnist.load_data()
        
        # Reescalando dados de [0 255] (greyscale) para [-1 1]
        X_train = (X_train - 127.5)/127.5
        X_train = np.expand_dims(X_train, axis=3)  #aumento de dimensao para casar com camadas convolucionais do modelo
        
        # Labels para falsos e verdadeiros
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        for epoch in range(epochs):
            
            # Treinamento do Discriminador
            
            # Seleciona um conjunto aleatorio de imagens
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            
            # Gera um conjunto de novas imagens
            gen_imgs = self.generator.predict(noise)
            
            # Treina o discriminador
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5*np.add(d_loss_fake, d_loss_real)
            
            # Treina Gerador
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            
            # Treina Gerador (usa indices como se dados criados fossem verdadeiros para induzir erro no discriminador)
            g_loss = self.combined.train_on_batch(noise, valid)
            
            # Plot do progresso do treinamento
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            
            # Salvando imagens geradas com certo intervalo de epocas
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                self.save_model()

            
    def sample_images(self, epoch):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r*c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        
        # Reescalando imagens para [0 1]
        gen_imgs = 0.5*gen_imgs + 0.5
        
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt+=1
        fig.savefig("images_gan_mlp/%d.png" % epoch)
        plt.close()
        
        
    def save_model(self):

        def save(model, model_name):
            model_path = "saved_model/%s.json" % model_name
            weights_path = "saved_model/%s_weights.hdf5" % model_name
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])

        save(self.generator, "generator")
        save(self.discriminator, "discriminator")
        
if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=14000, batch_size=32, sample_interval=200)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
________________________________________________

W0824 21:37:23.415326 18092 training.py:2197] Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
W0824 21:37:24.553685 18092 training.py:2197] Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?


0 [D loss: 0.486859, acc.: 54.69%] [G loss: 0.474694]
1 [D loss: 0.350700, acc.: 73.44%] [G loss: 0.561156]
2 [D loss: 0.335824, acc.: 85.94%] [G loss: 0.622399]
3 [D loss: 0.338018, acc.: 81.25%] [G loss: 0.729126]
4 [D loss: 0.294672, acc.: 90.62%] [G loss: 0.969579]
5 [D loss: 0.245872, acc.: 98.44%] [G loss: 1.124621]
6 [D loss: 0.199953, acc.: 98.44%] [G loss: 1.384585]
7 [D loss: 0.163131, acc.: 98.44%] [G loss: 1.576385]
8 [D loss: 0.122688, acc.: 100.00%] [G loss: 1.743874]
9 [D loss: 0.098521, acc.: 100.00%] [G loss: 2.027722]
10 [D loss: 0.082279, acc.: 100.00%] [G loss: 2.187666]
11 [D loss: 0.069155, acc.: 100.00%] [G loss: 2.338039]
12 [D loss: 0.060765, acc.: 100.00%] [G loss: 2.478453]
13 [D loss: 0.062806, acc.: 100.00%] [G loss: 2.554003]
14 [D loss: 0.062800, acc.: 100.00%] [G loss: 2.814813]
15 [D loss: 0.039146, acc.: 100.00%] [G loss: 2.878784]
16 [D loss: 0.034382, acc.: 100.00%] [G loss: 3.093394]
17 [D loss: 0.028977, acc.: 100.00%] [G loss: 3.121459]
18 [D loss

152 [D loss: 0.064040, acc.: 98.44%] [G loss: 3.593071]
153 [D loss: 0.094982, acc.: 96.88%] [G loss: 3.359643]
154 [D loss: 0.057493, acc.: 100.00%] [G loss: 3.544270]
155 [D loss: 0.077519, acc.: 98.44%] [G loss: 3.164924]
156 [D loss: 0.043098, acc.: 100.00%] [G loss: 3.412080]
157 [D loss: 0.140231, acc.: 95.31%] [G loss: 3.499435]
158 [D loss: 0.119368, acc.: 96.88%] [G loss: 3.739081]
159 [D loss: 0.145896, acc.: 93.75%] [G loss: 3.614543]
160 [D loss: 0.061952, acc.: 100.00%] [G loss: 3.296454]
161 [D loss: 0.153461, acc.: 93.75%] [G loss: 3.937261]
162 [D loss: 0.272029, acc.: 89.06%] [G loss: 2.382270]
163 [D loss: 0.086283, acc.: 96.88%] [G loss: 2.914068]
164 [D loss: 0.082921, acc.: 98.44%] [G loss: 3.501004]
165 [D loss: 0.085750, acc.: 98.44%] [G loss: 3.801023]
166 [D loss: 0.224450, acc.: 87.50%] [G loss: 3.154968]
167 [D loss: 0.060354, acc.: 98.44%] [G loss: 3.449807]
168 [D loss: 0.116378, acc.: 100.00%] [G loss: 3.348389]
169 [D loss: 0.117074, acc.: 98.44%] [G loss

299 [D loss: 0.729921, acc.: 43.75%] [G loss: 0.832730]
300 [D loss: 0.756627, acc.: 37.50%] [G loss: 0.729076]
301 [D loss: 0.688893, acc.: 45.31%] [G loss: 0.810053]
302 [D loss: 0.685131, acc.: 51.56%] [G loss: 0.803437]
303 [D loss: 0.673159, acc.: 46.88%] [G loss: 0.865509]
304 [D loss: 0.774865, acc.: 35.94%] [G loss: 0.772588]
305 [D loss: 0.642061, acc.: 50.00%] [G loss: 0.818788]
306 [D loss: 0.722337, acc.: 42.19%] [G loss: 0.838186]
307 [D loss: 0.789757, acc.: 40.62%] [G loss: 0.785225]
308 [D loss: 0.756162, acc.: 40.62%] [G loss: 0.721298]
309 [D loss: 0.735577, acc.: 40.62%] [G loss: 0.769409]
310 [D loss: 0.709829, acc.: 48.44%] [G loss: 0.747708]
311 [D loss: 0.728834, acc.: 46.88%] [G loss: 0.733877]
312 [D loss: 0.727143, acc.: 42.19%] [G loss: 0.731670]
313 [D loss: 0.697413, acc.: 48.44%] [G loss: 0.718619]
314 [D loss: 0.672192, acc.: 50.00%] [G loss: 0.781838]
315 [D loss: 0.721586, acc.: 45.31%] [G loss: 0.767857]
316 [D loss: 0.670868, acc.: 48.44%] [G loss: 0.

...

13407 [D loss: 0.660050, acc.: 59.38%] [G loss: 1.034838]
13408 [D loss: 0.604962, acc.: 73.44%] [G loss: 1.037049]
13409 [D loss: 0.692384, acc.: 62.50%] [G loss: 0.956515]
13410 [D loss: 0.568572, acc.: 64.06%] [G loss: 0.987925]
13411 [D loss: 0.612824, acc.: 67.19%] [G loss: 0.936556]
13412 [D loss: 0.648319, acc.: 56.25%] [G loss: 0.906764]
13413 [D loss: 0.657289, acc.: 70.31%] [G loss: 0.968518]
13414 [D loss: 0.698332, acc.: 46.88%] [G loss: 0.932273]
13415 [D loss: 0.605148, acc.: 70.31%] [G loss: 0.974797]
13416 [D loss: 0.621366, acc.: 68.75%] [G loss: 0.914792]
13417 [D loss: 0.647330, acc.: 60.94%] [G loss: 0.949982]
13418 [D loss: 0.694959, acc.: 56.25%] [G loss: 0.996797]
13419 [D loss: 0.628734, acc.: 67.19%] [G loss: 1.146461]
13420 [D loss: 0.620329, acc.: 75.00%] [G loss: 0.903444]
13421 [D loss: 0.655084, acc.: 64.06%] [G loss: 1.048943]
13422 [D loss: 0.605247, acc.: 68.75%] [G loss: 0.891202]
13423 [D loss: 0.627053, acc.: 73.44%] [G loss: 0.905714]
13424 [D loss:

13551 [D loss: 0.717079, acc.: 57.81%] [G loss: 0.964754]
13552 [D loss: 0.657660, acc.: 64.06%] [G loss: 1.033407]
13553 [D loss: 0.662933, acc.: 59.38%] [G loss: 1.025998]
13554 [D loss: 0.638516, acc.: 57.81%] [G loss: 0.918240]
13555 [D loss: 0.610177, acc.: 65.62%] [G loss: 0.928194]
13556 [D loss: 0.649204, acc.: 60.94%] [G loss: 0.943635]
13557 [D loss: 0.642961, acc.: 64.06%] [G loss: 0.984107]
13558 [D loss: 0.667449, acc.: 62.50%] [G loss: 1.058290]
13559 [D loss: 0.645768, acc.: 67.19%] [G loss: 1.004669]
13560 [D loss: 0.555282, acc.: 75.00%] [G loss: 1.041935]
13561 [D loss: 0.643915, acc.: 57.81%] [G loss: 0.975227]
13562 [D loss: 0.677494, acc.: 62.50%] [G loss: 1.066492]
13563 [D loss: 0.628035, acc.: 60.94%] [G loss: 1.036950]
13564 [D loss: 0.579322, acc.: 68.75%] [G loss: 1.056667]
13565 [D loss: 0.685488, acc.: 59.38%] [G loss: 0.990344]
13566 [D loss: 0.679393, acc.: 60.94%] [G loss: 0.996243]
13567 [D loss: 0.643776, acc.: 57.81%] [G loss: 1.021535]
13568 [D loss:

13693 [D loss: 0.615720, acc.: 59.38%] [G loss: 1.048738]
13694 [D loss: 0.616130, acc.: 60.94%] [G loss: 0.955391]
13695 [D loss: 0.640253, acc.: 67.19%] [G loss: 1.007647]
13696 [D loss: 0.662402, acc.: 60.94%] [G loss: 0.992634]
13697 [D loss: 0.586777, acc.: 76.56%] [G loss: 0.894553]
13698 [D loss: 0.619817, acc.: 65.62%] [G loss: 1.019723]
13699 [D loss: 0.601363, acc.: 65.62%] [G loss: 0.991203]
13700 [D loss: 0.662227, acc.: 56.25%] [G loss: 1.084915]
13701 [D loss: 0.716968, acc.: 60.94%] [G loss: 0.980708]
13702 [D loss: 0.630879, acc.: 67.19%] [G loss: 1.053024]
13703 [D loss: 0.667258, acc.: 57.81%] [G loss: 0.924035]
13704 [D loss: 0.604939, acc.: 70.31%] [G loss: 0.967053]
13705 [D loss: 0.610995, acc.: 70.31%] [G loss: 1.049603]
13706 [D loss: 0.688921, acc.: 62.50%] [G loss: 1.073361]
13707 [D loss: 0.594820, acc.: 71.88%] [G loss: 0.947049]
13708 [D loss: 0.568485, acc.: 70.31%] [G loss: 1.005254]
13709 [D loss: 0.588599, acc.: 67.19%] [G loss: 0.948138]
13710 [D loss:

13836 [D loss: 0.640546, acc.: 67.19%] [G loss: 1.050133]
13837 [D loss: 0.690940, acc.: 65.62%] [G loss: 0.912854]
13838 [D loss: 0.690372, acc.: 56.25%] [G loss: 1.062429]
13839 [D loss: 0.682427, acc.: 56.25%] [G loss: 1.089849]
13840 [D loss: 0.631456, acc.: 64.06%] [G loss: 0.987198]
13841 [D loss: 0.736694, acc.: 53.12%] [G loss: 1.052964]
13842 [D loss: 0.605430, acc.: 67.19%] [G loss: 1.038703]
13843 [D loss: 0.644560, acc.: 64.06%] [G loss: 1.090669]
13844 [D loss: 0.634132, acc.: 65.62%] [G loss: 0.975899]
13845 [D loss: 0.615559, acc.: 62.50%] [G loss: 1.076615]
13846 [D loss: 0.588079, acc.: 68.75%] [G loss: 1.000407]
13847 [D loss: 0.618797, acc.: 67.19%] [G loss: 1.036101]
13848 [D loss: 0.647443, acc.: 59.38%] [G loss: 0.949583]
13849 [D loss: 0.749856, acc.: 54.69%] [G loss: 1.030875]
13850 [D loss: 0.598906, acc.: 68.75%] [G loss: 0.991634]
13851 [D loss: 0.649742, acc.: 65.62%] [G loss: 0.979913]
13852 [D loss: 0.599222, acc.: 73.44%] [G loss: 1.079046]
13853 [D loss:

13983 [D loss: 0.653133, acc.: 62.50%] [G loss: 1.032062]
13984 [D loss: 0.618678, acc.: 65.62%] [G loss: 1.111734]
13985 [D loss: 0.640875, acc.: 65.62%] [G loss: 0.981948]
13986 [D loss: 0.592885, acc.: 70.31%] [G loss: 0.950659]
13987 [D loss: 0.694179, acc.: 60.94%] [G loss: 0.905336]
13988 [D loss: 0.676818, acc.: 59.38%] [G loss: 1.002357]
13989 [D loss: 0.624035, acc.: 60.94%] [G loss: 0.923065]
13990 [D loss: 0.650142, acc.: 64.06%] [G loss: 0.946292]
13991 [D loss: 0.658908, acc.: 68.75%] [G loss: 0.892511]
13992 [D loss: 0.704283, acc.: 57.81%] [G loss: 0.946003]
13993 [D loss: 0.623191, acc.: 68.75%] [G loss: 1.058902]
13994 [D loss: 0.608852, acc.: 64.06%] [G loss: 0.989244]
13995 [D loss: 0.605047, acc.: 67.19%] [G loss: 1.004105]
13996 [D loss: 0.628998, acc.: 57.81%] [G loss: 0.908118]
13997 [D loss: 0.595406, acc.: 70.31%] [G loss: 0.916514]
13998 [D loss: 0.653007, acc.: 62.50%] [G loss: 0.942724]
13999 [D loss: 0.697532, acc.: 60.94%] [G loss: 0.902317]
